In [1]:
import pandas as pd

from lineups import FantasyTeam, Week

In [2]:
OWNERS = ["Oscar", "Lucy", "Curtis", "Diana", "Luis"]

roster_data = pd.read_csv("fantasy_rosters.csv")
rosters = [FantasyTeam(roster_data[roster_data["owner"] == o]) for o in OWNERS]

In [3]:
defaults = Week([r.default_lineup for r in rosters])
defaults.results_by_averages

(owner
 Curtis    22.0
 Diana     18.0
 Lucy      19.0
 Luis      14.0
 Oscar     17.0
 dtype: float64,
 owner     Curtis   Diana    Lucy    Luis   Oscar
 pts       119.34  115.01  110.67  107.84  120.78
 trb        40.57   35.36   44.22   48.11   31.38
 ast        35.99   23.95   26.24   23.82   24.63
 stl         5.08    7.96    5.96    5.06    5.83
 blk         4.54    4.63    3.90    4.94    3.76
 fg3        12.02    8.25    8.49    4.08   11.24
 pts_rank    4.00    3.00    2.00    1.00    5.00
 trb_rank    3.00    2.00    4.00    5.00    1.00
 ast_rank    5.00    2.00    4.00    1.00    3.00
 stl_rank    2.00    5.00    4.00    1.00    3.00
 blk_rank    3.00    4.00    2.00    5.00    1.00
 fg3_rank    5.00    2.00    3.00    1.00    4.00)

In [4]:
max_zs = Week([r.max_z_lineup for r in rosters])
max_zs.results_by_averages

(owner
 Curtis    19.0
 Diana     19.0
 Lucy      17.0
 Luis      16.0
 Oscar     19.0
 dtype: float64,
 owner     Curtis   Diana    Lucy   Luis   Oscar
 pts       126.49  112.85  110.67  99.33  123.58
 trb        38.14   39.17   44.22  47.70   31.87
 ast        37.89   23.05   26.24  23.84   24.46
 stl         5.55    7.57    5.96   6.23    6.33
 blk         2.79    6.06    3.90   4.95    4.02
 fg3        13.22    8.22    8.49   4.78   11.42
 pts_rank    5.00    3.00    2.00   1.00    4.00
 trb_rank    2.00    3.00    4.00   5.00    1.00
 ast_rank    5.00    1.00    4.00   2.00    3.00
 stl_rank    1.00    5.00    2.00   3.00    4.00
 blk_rank    1.00    5.00    2.00   4.00    3.00
 fg3_rank    5.00    2.00    3.00   1.00    4.00)

In [5]:
def get_delta(lineup, result, owner):
    scores = result[0]
    score = scores[owner]
    rank = scores.rank(ascending=False)[owner]
    if rank == 1.0:
        delta = score - scores.nlargest(2).min()
    else:
        delta = score - scores.max()
    return {
        "lineup": lineup,
        "score": score,
        "delta": delta,
        "rank": rank,
    }

In [6]:
top_lineups = {}

for i, r in enumerate(rosters):
    owner = r.owner
    all_possible_lineups = r.all_possible_lineups()

    others = rosters[:i] + rosters[(i + 1):]
    defaults = [o.default_lineup for o in others]
    max_zs = [o.max_z_lineup for o in others]

    default_results = []
    max_z_results = []

    for l in all_possible_lineups:
        d_result = Week(defaults + [l]).score_by_averages()
        mz_result = Week(max_zs + [l]).score_by_averages()

        default_results.append(get_delta(l, d_result, owner))
        max_z_results.append(get_delta(l, mz_result, owner))
        
    top_lineups[f"{owner}_default"] = sorted(default_results, key=lambda x: x["delta"], reverse=True)[:3]
    top_lineups[f"{owner}_max_z"] = sorted(max_z_results, key=lambda x: x["delta"], reverse=True)[:3]

In [7]:
top_lineups

{'Oscar_default': [{'lineup': Lineup([('CJ McCollum', 'G'), ('James Harden', 'G'), ('Paul George', 'F'), ('Zion Williamson', 'F'), ('LaMarcus Aldridge', 'C')]),
   'score': 19.0,
   'delta': -2.0,
   'rank': 2.0},
  {'lineup': Lineup([("De'Aaron Fox", 'G'), ('James Harden', 'G'), ('Brandon Ingram', 'F'), ('Zion Williamson', 'F'), ('LaMarcus Aldridge', 'C')]),
   'score': 19.0,
   'delta': -2.0,
   'rank': 2.0},
  {'lineup': Lineup([("De'Aaron Fox", 'G'), ('James Harden', 'G'), ('Brandon Ingram', 'F'), ('Paul George', 'F'), ('Zion Williamson', 'F')]),
   'score': 19.0,
   'delta': -3.0,
   'rank': 2.0}],
 'Oscar_max_z': [{'lineup': Lineup([("De'Aaron Fox", 'G'), ('James Harden', 'G'), ('Brandon Ingram', 'F'), ('Paul George', 'F'), ('Zion Williamson', 'F')]),
   'score': 19.0,
   'delta': 0.0,
   'rank': 2.0},
  {'lineup': Lineup([("De'Aaron Fox", 'G'), ('James Harden', 'G'), ('Brandon Ingram', 'F'), ('Paul George', 'F'), ('LaMarcus Aldridge', 'C')]),
   'score': 18.0,
   'delta': -1.0,


In [12]:
top_lineups_df = pd.DataFrame([
    {
        "owner": k.split("_")[0],
        "opposition": k.split("_")[1],
        "guard1": l["lineup"].players[0].name,
        "guard2": l["lineup"].players[1].name,
        "big1": l["lineup"].players[2].name,
        "big2": l["lineup"].players[3].name,
        "big3": l["lineup"].players[4].name,
        "score": l["score"],
        "delta": l["delta"],
        "place": l["rank"],
    }
    for k, lineups in top_lineups.items()
    for l in lineups
])
top_lineups_df.to_csv("top_lineups.csv")

In [19]:
r = rosters[2]
owner = r.owner
lineups_without_lebron = [
    l for l in r.all_possible_lineups()
    if all(p.name != "LeBron James" for p in l.players)
]

others = rosters[:2] + rosters[(2 + 1):]
defaults = [o.default_lineup for o in others]
max_zs = [o.max_z_lineup for o in others]

default_results = []
max_z_results = []

for l in lineups_without_lebron:
    d_result = Week(defaults + [l]).score_by_averages()
    mz_result = Week(max_zs + [l]).score_by_averages()

    default_results.append(get_delta(l, d_result, owner))
    max_z_results.append(get_delta(l, mz_result, owner))

no_lebron_default = sorted(default_results, key=lambda x: x["delta"], reverse=True)[:3]
no_lebron_max_z = sorted(max_z_results, key=lambda x: x["delta"], reverse=True)[:3]

In [24]:
[p.name for l in no_lebron_default for p in l["lineup"].players]

['Luka Doncic',
 'Damian Lillard',
 'Kyle Lowry',
 'Hassan Whiteside',
 'Joel Embiid',
 'Luka Doncic',
 'Damian Lillard',
 'Kyle Lowry',
 'Joel Embiid',
 'Jonathan Isaac',
 'Luka Doncic',
 'Kyle Lowry',
 'Kristaps Porzingis',
 'Lonzo Ball',
 'Jonathan Isaac']

In [25]:
[p.name for l in no_lebron_max_z for p in l["lineup"].players]

['Luka Doncic',
 'Damian Lillard',
 'Kyle Lowry',
 'Hassan Whiteside',
 'Kristaps Porzingis',
 'Luka Doncic',
 'Damian Lillard',
 'Kyle Lowry',
 'Hassan Whiteside',
 'Joel Embiid',
 'Luka Doncic',
 'Damian Lillard',
 'Kyle Lowry',
 'Kristaps Porzingis',
 'Joel Embiid']